In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
# stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [6]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (1.5,1.5,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(-0.2,-0.5,0.08), rpy=(0,0,np.pi/2), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), center=(0,0,-0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_handle")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_action import Gripper2Tool, PlacePlane

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_object import CustomObject, Grasp2Point, PlacePoint

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush = pscene.create_object(oname="brush", gname="brush_handle", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]), 
                                                   "face": PlacePoint("face", brush_handle, [0,0,-0.075], [0,0,0])})

### planning pipeline

In [15]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## ui

In [16]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Test plan

In [17]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

obj, handle, binder = "brush", "face", "goal_bd"
initial_state = pscene.get_state(crob.home_pose)
goal_nodes = pscene.get_goal_nodes(initial_state.node, obj, binder)
ppline.search(initial_state, goal_nodes, verbose=True, display=True, dt_vis=0.01)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

node: ('floor',)->('grip0',) = fail
node: ('floor',)->('grip0',) = fail
node: ('floor',)->('grip0',) = fail
node: ('floor',)->('grip0',) = fail
node: ('floor',)->('grip0',) = fail
node: ('floor',)->('grip0',) = success
Remaining:2->1 / branching: 0->1 (1.37/600.0 s, steps/err: 36(689.303874969 ms)/2.32863151515)
node: ('grip0',)->('goal',) = success
Remaining:1->0 / branching: 1->2 (2.53/600.0 s, steps/err: 55(1151.44085884 ms)/0.00139075504925)
======================= terminated 0: first answer acquired ===============================


In [18]:
schedule = ppline.idxSchedule2SnodeScedule(ppline.find_schedules().values()[0])

In [19]:
ppline.play_schedule(schedule)

In [20]:
print(schedule[0].state.node)
print(schedule[1].state.node)
print(schedule[2].state.node)

(('brush', 'face', 'floor'),)
(('brush', 'handle', 'grip0'),)
(('brush', 'face', 'goal_bd'),)


In [21]:
idx = 0
from_state = schedule[idx].state
to_state = schedule[idx+1].state
redundancy_dict = schedule[idx+1].redundancy_dict
redundancy_dict[obj]['w'] = 0
traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                    redundancy_dict=redundancy_dict)
print(success)
if success:
    for bd in binding_list:
        pscene.rebind(bd, list2dict(lastQ, pscene.gscene.joint_names))

True


In [22]:
new_state = to_state.copy(pscene)
new_state.obj_pos_dict = pscene.get_object_state()[1]
new_state.Q = lastQ

In [23]:
gscene.show_motion(traj)

In [24]:
idx = 1
from_state = new_state.copy(pscene)
to_state = from_state.copy(pscene)
to_state.node = (('brush', 'face', 'goal_bd'),)
redundancy_dict = schedule[idx+1].redundancy_dict
redundancy_dict = {obj:{'w': np.pi/2}}
traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                    redundancy_dict=redundancy_dict)
print(success)

True


In [25]:
gscene.show_motion(traj, period=0.05)

In [26]:
from pkg.planning.motion.moveit.moveit_planner import make_constraint_list

##
# @class MotionManifold
# @brief geometry manifold to constrain motion
class MotionManifold:
    ##
    # @param geometry_list list of rnb-planning.src.pkg.geometry.geometry.GeometryItem
    def __init__(self, geometry_list):
        self.geometry_list = geometry_list
        
        
##
# @brief add motion constraint for moveit planner
# @param
def add_constraint(self, group_name, tool_link, tool_offset_T, manifold, fix_surface, fix_normal, tol=1e-3, use_box=False):
    constraint_manifold_list = make_constraint_list(manifold.geometry_list, use_box=use_box)
    xyzquat = T2xyzquat(tool_offset_T)
    self.planner.add_union_manifold_py(group_name=group_name, tool_link=tool_link, tool_offset=xyzquat[0]+xyzquat[1], 
                                       geometry_list=constraint_manifold_list, fix_surface=fix_surface, fix_normal=fix_normal, tol=tol)

In [27]:
from pkg.planning.constraint.constraint_common import calc_redundancy
self = mplan
if len(binding_list)!=1:
    raise(RuntimeError("Only single manipulator operation is implemented with moveit!"))
self.update_gscene()

obj_name, ap_name, binder_name = binding_list[0]
redundancy = redundancy_dict[obj_name] if redundancy_dict else None

binder = self.pscene.binder_dict[binder_name]
obj = self.pscene.object_dict[obj_name]
handle = obj.action_points_dict[ap_name]
point_add, rpy_add = calc_redundancy(redundancy, binder)
T_handle = handle.Toff_lh
T_binder = np.matmul(binder.Toff_lh, SE3(Rot_rpy(rpy_add), point_add))

if len(self.planner.group_names)==1:
    group_name_handle = self.planner.group_names if handle.geometry.link_name in self.urdf_content.parent_map else []
    group_name_binder = self.planner.group_names if binder.geometry.link_name in self.urdf_content.parent_map else []
else:
    group_name_handle = [gname for gname in self.planner.group_names if gname in handle.geometry.link_name]
    group_name_binder = [gname for gname in self.planner.group_names if gname in binder.geometry.link_name]

dual = False
if group_name_binder and not group_name_handle:
    group_name = group_name_binder[0]
    tool, T_tool = binder, T_binder
    target, T_tar = handle, T_handle
elif group_name_handle and not group_name_binder:
    group_name = group_name_handle[0]
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder
else:
    if not self.enable_dual:
        raise(RuntimeError("dual arm motion is not enabled"))
    dual = True
    group_name = "{}_{}".format(group_name_binder[0], group_name_handle[0])
    self.update_gscene(group_name)
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder

T_tar_tool = np.matmul(T_tar, SE3_inv(T_tool))
goal_pose = tuple(T_tar_tool[:3,3]) \
            +tuple(Rotation.from_dcm(T_tar_tool[:3,:3]).as_quat())

if dual:
    dual_planner = self.dual_planner_dict[group_name]
    planner = dual_planner.planner
    from_Q = from_state.Q[dual_planner.idx_pscene_to_mpc]*dual_planner.joint_signs
else:
    planner = self.planner
    if self.need_mapping:
        from_Q = from_state.Q[self.idx_pscene_to_mpc]
    else:
        from_Q = from_state.Q

In [28]:
# manifold = MotionManifold([gscene.NAME_DICT["floor"]])

In [29]:
gscene.show_pose(from_Q)

In [30]:
floor = gscene.NAME_DICT["floor"]
floor_top = gscene.copy_from(floor, new_name="floor_top", color=(0,1,0,0.8), display=True, collision=False)
floor_top.set_offset_tf(center=tuple(np.add(floor_top.center, (0,0,floor_top.dims[2]/2))), 
                        orientation_mat=np.matmul(floor_top.orientation_mat, Rot_axis(1,0)))
floor_top.set_dims(floor_top.dims[:2]+(0.001,))

In [31]:
T_goal = T_xyzquat((goal_pose[:3], goal_pose[3:]))
goal_pose_marker = gscene.create_safe(name="goal_pose", gtype=GEOTYPE.SPHERE, link_name="base_link",
                   dims=(0.01, 0.01, 0.01), center=tuple(T_goal[:3,3]), rpy=Rot2rpy(T_goal[:3,:3]), collision=False, display=True,
                   color=(1,0,0,1))

In [32]:
gscene.set_rviz()

In [33]:
gscene.add_highlight_axis("ax", "goal_pose", goal_pose_marker.link_name, goal_pose_marker.center, goal_pose_marker.orientation_mat)

In [34]:
gscene.show_pose(lastQ)
print(lastQ)

[ 0.73395835 -0.56668788 -1.72362916 -0.00175243 -0.85118597  0.73464737]


In [35]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
# manifold = MotionManifold([floor_top])
# add_constraint(mplan, group_name, tool.geometry.link_name, tool.Toff_lh, manifold, fix_surface=True, fix_normal=True, tol=1e-3)
manifold = MotionManifold([floor])
add_constraint(mplan, group_name, tool.geometry.link_name, tool.Toff_lh, manifold, fix_surface=True, fix_normal=False, tol=5e-3, 
               use_box=True)

In [36]:
trajectory, success = mplan.planner.plan_constrained_py(
    group_name,tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q), timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (33)


In [37]:
gscene.show_motion(trajectory, period=0.05)

In [38]:
print(np.round(from_Q,2))

[-0.03 -0.57 -1.72  0.   -0.85 -0.03]


In [39]:
x = str2num_split_blank("-0.580574  -0.546569   -1.22459  -0.587711   -0.45868 -0.0399314")
x_dict = list2dict(x, crob.joint_names)

In [40]:
tool.geometry.get_tf(x_dict)[:3,3]

array([-0.69601878, -0.61164497,  0.51541016])

## Disconnect stereo

In [26]:
stereo.disconnnect()